In [1]:
# Import the requests library.

import numpy as np 
from citipy import citipy
import gmaps
import requests
import pandas as pd 
from datetime import datetime

from config import weather_api_key




# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)
# Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"
print(city_url)


http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=a992c33c6933d1294ff22aff80a2e12b
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=a992c33c6933d1294ff22aff80a2e12b&q=Boston


In [2]:
#Create a new set of 2,000 random latitudes and longitudes.
lats = np.random.uniform(-90.0,90.0, size=2000)
lngs = np.random.uniform(-180.0,180.0,size=2000)
lats_lngs = zip(lats, lngs)
coordinates = list(lats_lngs)

In [3]:
# Identify the nearest city for each latitude and longitude combination.
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name 
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city) 
# Loop through all the cities in our list.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1 
for i in range(len(cities)):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i] 

Beginning Data Retrieval     
-----------------------------


In [4]:
# Loop through all the cities in the list.
for i, city in enumerate(cities): 

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1 
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_description = str(city_weather["weather"][0]["description"]) 
        city_max_temp = city_weather["main"]["temp_max"] 
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"] 
    
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Current Description": city_description,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 15 | mataura
Processing Record 2 of Set 15 | butaritari
Processing Record 3 of Set 15 | ushuaia
Processing Record 4 of Set 15 | mungaa
Processing Record 5 of Set 15 | khatanga
Processing Record 6 of Set 15 | vaini
Processing Record 7 of Set 15 | severo-kurilsk
Processing Record 8 of Set 15 | jamestown
Processing Record 9 of Set 15 | koslan
Processing Record 10 of Set 15 | torbay
Processing Record 11 of Set 15 | menongue
Processing Record 12 of Set 15 | kandrian
Processing Record 13 of Set 15 | umzimvubu
City not found. Skipping...
Processing Record 14 of Set 15 | tasiilaq
Processing Record 15 of Set 15 | los llanos de aridane
Processing Record 16 of Set 15 | new norfolk
Processing Record 17 of Set 15 | hilo
Processing Record 18 of Set 15 | marsa matruh
Processing Record 19 of Set 15 | port hedland
Processing Record 20 of Set 15 | fortuna
Processing Record 21 of Set 15 | luena
Processing Record 22 of Set 15 | hokitika
Processing Record 23 of Set 15 | albany
Pr

In [5]:
city_data_df = pd.DataFrame(city_data)
 
city_data_reordered = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_data_df = city_data_df[city_data_reordered] 

city_data_df.to_csv('WeatherPy_database.csv') 